# Visualize the data
1. Convenience functions for date processing
2. Get trips that are currently active
3. Get status of the active trips

In [3]:
from pandas import read_csv, DataFrame
from os import path, getcwd

gtfs_filtered_path = path.join(getcwd(), 'gtfs_filtered')
calendar_path = path.join(gtfs_filtered_path, 'calendar.txt')
routes_path = path.join(gtfs_filtered_path, 'routes.txt')
trips_path = path.join(gtfs_filtered_path, 'trips.txt')
stops_path = path.join(gtfs_filtered_path, 'stops.txt')
stop_times_path = path.join(gtfs_filtered_path, 'stop_times.txt')

calendar:DataFrame = read_csv(calendar_path)
routes:DataFrame = read_csv(routes_path)
trips:DataFrame = read_csv(trips_path)
stops:DataFrame = read_csv(stops_path)
stop_times:DataFrame = read_csv(stop_times_path)





## 1. convenience functions for gtfs date formats

In [5]:
import datetime

def parseTimeAsDatetimeObject(timestring:str):
    # mod 24, because gtfs defines days as service days that can be longer than 24 hours, so 24:15 is a valid gtfs time
    hour = int(timestring[0:2]) % 24
    minute = int(timestring[3:5])
    second = int(timestring[6:8])
    #print(timestring)
    #print(hour)
    #print(minute) 
    #print(second)
    return datetime.time(hour, minute, second)

def parseDateAsDatetimeObject(datestring:str):
    datestring = str(datestring)
    year = int(datestring[0:4])
    month = int(datestring[4:6])
    day = int(datestring[6:8])
    return datetime.date(year, month, day)

def getGtfsWeekdayFromDate(date: datetime.date):
    weekday_number = date.weekday()
    if weekday_number == 0:
        return "monday"
    elif weekday_number == 1:
        return "tuesday"
    elif weekday_number == 2:
        return "wednesday"
    elif weekday_number == 3:
        return "thursday"
    elif weekday_number == 4:
        return "friday"
    elif weekday_number == 5:
        return "saturday"
    else:
        return "sunday"
    

## 2. currently active trips

First, we need to get all the trip_ids for currently active trips. Trips are active, if the current time is between the start and end time of the trip and if one of the services, the trip belongs to, runs on the current day.
Let's start by looking at the start and end times of the trips.

In [8]:
print(datetime.datetime.now())

def isTripRowActiveAtCurrentTime(trip_row):
    start_time = parseTimeAsDatetimeObject(trip_row['start_time'])
    current_time = datetime.datetime.now().time() 
    end_time = parseTimeAsDatetimeObject(trip_row['end_time'])
    #print(start_time, current_time, end_time, start_time <= current_time <= end_time)
    return start_time <= current_time <= end_time
    

# select trips where current time is between start and end time
trips = trips[trips.apply(isTripRowActiveAtCurrentTime, axis=1)]
print("found", trips.shape[0], "trips that run at the current time")
print(trips.head(5))

2024-10-21 18:40:01.949105
found 10 trips that run at the current time
      route_id            trip_id  \
54     22-2-22    22-2-1022-66780   
73     22-1-22       22-1-1-66120   
93     22-1-22       22-1-1-66720   
176    22-2-22    22-2-1022-66180   
264  22-852-22  22-852-3022-66780   

                                            service_id  trip_short_name  \
54         295-296-297-298-299-300-302-303-304-305-307               22   
73   295-296-297-298-299-300-301-302-303-304-305-30...               22   
93         295-296-297-298-299-300-302-303-304-305-307               22   
176        295-296-297-298-299-300-302-303-304-305-307               22   
264                                                309               22   

    start_time  end_time  
54    18:33:00  18:52:00  
73    18:22:00  18:41:00  
93    18:32:00  18:51:00  
176   18:23:00  18:42:00  
264   18:33:00  19:01:00  


Secondly, we will check whether the services run on the current day by looking up the services from the `service_id` column in the calendar dataframe.
As soon as we find a `service_id` that runs on the current day, we can stop the search and return true, otherwise we return false.

In [17]:
def isTripRowActiveOnCurrentDay(trip_row):
    current_date = datetime.date.today()
    current_weekday_gtfs = getGtfsWeekdayFromDate(datetime.date.today())

    
    calendar:DataFrame = read_csv(calendar_path)

    # select row from calendar for this service
    calendar = calendar[calendar['service_id'] == trip_row['service_id']]

    # check every calendar entry
    for index, schedule in calendar.iterrows():
        # check if current date is between start_date and end_date (inclusive)
        start_date = parseDateAsDatetimeObject(schedule['start_date'])
        end_date = parseDateAsDatetimeObject(schedule['end_date'])

        duration_check = start_date <= current_date <= end_date

        # check if current weekday is an active day in the schedule
        weekday_check = schedule[current_weekday_gtfs] == 1

        if duration_check and weekday_check:
            return True
                
    return False
    
trips = trips[trips.apply(isTripRowActiveOnCurrentDay, axis=1)]
print(trips.head(5))

    route_id          trip_id  \
54   22-2-22  22-2-1022-66780   
73   22-1-22     22-1-1-66120   
93   22-1-22     22-1-1-66720   
176  22-2-22  22-2-1022-66180   

                                            service_id  trip_short_name  \
54         295-296-297-298-299-300-302-303-304-305-307               22   
73   295-296-297-298-299-300-301-302-303-304-305-30...               22   
93         295-296-297-298-299-300-302-303-304-305-307               22   
176        295-296-297-298-299-300-302-303-304-305-307               22   

    start_time  end_time  
54    18:33:00  18:52:00  
73    18:22:00  18:41:00  
93    18:32:00  18:51:00  
176   18:23:00  18:42:00  


## Status of active trips
Now that we have identified all the trips that are currently running, we want to know where the trams are on our network. As we later want to represent a vehicle being at a stop as well as a vehicle traveling between stops, we will represent the status of a vehicle (trip) as 
status: IN_TRANSIT_TO, currentStop: <stop_id>, previousStop: <stop_id>
status: STOPPED_AT, currentStop: <stop_id>, previousStop: <stop_id>
and optionally (status: INCOMING_AT, stop: <stop>)

Additionally, we want to represent the direction of travel by having one LED line per direction. All stops of each section of track between intersections will be organized in a sorted list for each direction, e,g, for the section between Bismarckplatz and Stadtbücherei
List 1: None -> Bismarckplatz -> Seegarten -> Stadtbücherei
List 2: None -> Stadtbücherei -> Seegarten -> Bismarckplatz

Each tuple of each direction will have LEDs assigned, e.g. Bismarckplatz -> Seegarten gets LEDs 0 and 1 assigned. 0 is the LED between Bismarckplatz and Seegarten and 1 is the LED on Seegarten itself. When the status reflects the train having stopped, LED 1 is lighted, if the tram is still in transit to Seegarten, the LED between the stations is activated instead.  
